In [ ]:
import os
import os.path
import time
import requests
from bs4 import BeautifulSoup as bsoup

In [ ]:
def download_file(local_filename, download_link, username, password):
    '''
    Logs into IMSLP and downloads a single file.
    '''
    time.sleep(1)
    login_url = 'https://imslp.org/index.php?title=Special:UserLogin&returnto=Main%20Page'
    with requests.Session() as s:
        r = s.get(login_url)
        cookies = dict(r.cookies)
        response = r.content
        soup = bsoup(response,"html.parser")
        token = soup.find_all("input",{"name":"wpLoginToken"})[0]['value']
        payload = {
            'wpName': username,
            'wpPassword': password,
            'wpLoginAttempt':'Log in',
            'wpLoginToken': token
        }
        login_url = 'https://imslp.org/index.php?title=Special:UserLogin&action=submitlogin&type=login&returnto=Main%20Page'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        p = s.post(login_url, data = payload, cookies=cookies)
        #print(p.text)
        req = s.get(download_link)
        with open(local_filename, 'wb') as f:
            for chunk in req.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
            return local_filename

In [ ]:
def download_all_filler(download_list, username, password):
    '''
    Downloads all of the filler scores in the database.
    '''
    with open(download_list, 'r') as f:
        for line in f:
            parts = line.strip().split()
            assert len(parts) == 2
            local_filename = parts[0]
            download_link = parts[1]
            print('Downloading {}'.format(os.path.basename(local_filename)))
            download_file(local_filename, download_link, username, password)

In [ ]:
download_list = 'test.txt'
imslp_username = 'username' # replace with valid username & password
imslp_password = 'password'
download_all_filler(download_list, imslp_username, imslp_password)